In [20]:
from openai import OpenAI

my_api_key = "sk-zgUvoFsKaZMh0N-hz-c0Pw"
token_limit = 131000  # Token limit for the prompt

# OpenAI client
openai_client = OpenAI(base_url="https://llm.scads.ai/v1", api_key=my_api_key)

# Find model with "llama" in name
for model in openai_client.models.list().data:
    model_name = model.id
    if "Qwen/" in model_name:
        print(model_name)
        break


Qwen/Qwen2-VL-7B-Instruct


In [17]:
from PIL import Image

def encode_image(image_path, max_size=(512, 512), quality=75, convert_to_jpeg=True):
    """
    Resize and compress an image before encoding it to Base64.
    
    Parameters:
    - image_path (str): Path to the image.
    - max_size (tuple): Maximum width and height (default: 512x512).
    - quality (int): JPEG quality (1-100), lower = smaller size.
    - convert_to_jpeg (bool): Convert PNG to JPEG to reduce size.

    Returns:
    - str: Base64 encoded string of the optimized image.
    """
    with Image.open(image_path) as img:
        # Convert PNG to JPEG (optional)
        if convert_to_jpeg and img.format == "PNG":
            img = img.convert("RGB")  # Remove alpha channel for JPEG

        # Resize image while maintaining aspect ratio
        img.thumbnail(max_size, Image.Resampling.LANCZOS)  # Use high-quality resizing

        # Save to a BytesIO buffer
        img_buffer = io.BytesIO()
        img_format = "JPEG" if convert_to_jpeg else img.format  # Save as JPEG if converting
        img.save(img_buffer, format=img_format, quality=quality, optimize=True)

        # Convert to Base64
        img_buffer.seek(0)
        return base64.b64encode(img_buffer.getvalue()).decode("utf-8")


In [14]:
def get_image_paths(folder):
    image_paths = sorted([
        os.path.join(folder, f) for f in os.listdir(folder) if f.endswith(".png")
    ])
    return image_paths

# Example usage
image_folder = "downloaded_images"
image_files = get_image_paths(image_folder)

In [43]:
def ask_vlm_one_by_one(image_paths, knowledge_level="intermediate"):    
    for model in openai_client.models.list().data:
        model_name = model.id
        if "Qwen/" in model_name:
            break
        
    system_prompt = f"You are an AI assistant that analyzes slide presentations and creates a set of Exam Questions from them. Formulate the questions depending on the knowledge level, to make it easier or more detailed. The level is {knowledge_level}."
    prompt = "Take a look at the Slide and suggest a Exam Question for College Students concerning the topic of the current Slide. Output ONLY the Question, no additional information or explanations. Also output EXACTELY one Question per Image."
    
    responses = [] 

    for img_path in image_paths:
        base64_image = encode_image(img_path)  # Convert image to Base64

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": [
                {"type": "text", "text": prompt},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
            ]}
        ]

        # Send request
        response = openai_client.chat.completions.create(
            model=model_name,
            messages=messages
        )

        # Store response
        responses.append(response.choices[0].message.content)

    return responses 



In [44]:
responses = ask_vlm_one_by_one(image_files, "beginner")
for i, res in enumerate(responses):
    print(f"Response for Image {i+1}: {res}")

Response for Image 1: What challenges do researchers face in managing large datasets?
Response for Image 2: What are the key aspects that should be considered during the RDM life cycle for research data management?
Response for Image 3: What is the impact of licensing on the RDM life cycle and how does it affect future cycles or acquisitions?
Response for Image 4: "Define the key components of good Research Data Management (RDM) in terms of responsibilities, processes, communication, personnel, and expertise."
Response for Image 5: Identify the key differences between a domain specialist, data analyst, and IT specialist roles, focusing on their primary responsibilities and related examples given in the slide.
Response for Image 6: How do Data Management Plans (DMPs) describe the IS-state of a data environment and what aspects are targeted in a DMP?
Response for Image 7: What are the key responsibilities and procedures that should be defined early in the Open Science project according t

In [46]:
def ask_for_summary(responses_per_slide):
    openai_client = OpenAI(base_url="https://llm.scads.ai/v1", api_key=my_api_key)

    # Find model with "llama" in name
    for model in openai_client.models.list().data:
        model_name = model.id
        if "Llama" in model_name:
            break
    
    number_of_slides = len(responses_per_slide)
    max_questions = number_of_slides / 3
    
    questions = ", ".join(responses_per_slide)
    
    prompt = f"""
    Here is a list of Exam Questions : {questions}, relating to one specific deck of presentation Slides. 
    Each Slide got converted into a question. Go through the list and extract up to {max_questions} reasonable Exam Questions for College Students to that specific topic.
    Output the Questions numerated and output nothing else than those questions (no extra explanation, etc.):
    1. Question 1
    2. Questions 2
    ...

    Try to stick as closely to the topics from the list as possible and extract the key points into new questions.
    """
    
    messages = [
            {"role": "user", "content": [
                {"type": "text", "text": prompt}
            ]}
        ]

    # Send request
    response = openai_client.chat.completions.create(model=model_name,messages=messages)

    return response.choices[0].message.content

In [47]:
summary = ask_for_summary(responses)
print(summary)

1. What are the key challenges in managing large datasets in research?
2. How does licensing impact the research data management life cycle?
3. What are the main components of good Research Data Management (RDM)?
4. What is the difference between a domain specialist, data analyst, and IT specialist in research?
5. What is the purpose of a Data Management Plan (DMP) in research projects?
6. What are the key responsibilities of a Data Steward in research data management?
7. What is the role of a data scientist in research and what skills are required?
8. How can software developed by researchers be maintained and supported after the project is completed?
9. What are the benefits of sharing and licensing research data and results?
10. What is the difference between reliability and reproducibility in scientific experiments?
11. How can researchers ensure that their research results are reusable and accessible?
12. What are the FAIR principles for sharing and managing research data?
13. Wha

In [49]:
# Test if changing the knowledge level works
responses = ask_vlm_one_by_one(image_files, "expert")
summary = ask_for_summary(responses)

In [50]:
print(summary)

1. How does Research Data Management contribute to the scalability and efficiency of data analytics in the context of artificial intelligence?
2. What are the key components of the RDM life cycle and what is their sequence?
3. What are the three key aspects of the RDM Life Cycle and how do they contribute to the overall lifecycle of data management?
4. What are the key components that constitute good Research Data Management (RDM)?
5. What is the purpose of Data Management Plans (DMPs) in the context of data environments?
6. How can Data Management Plans (DMPs) be used to ensure the successful completion of a scientific project?
7. What is the difference between archiving and backup when regularly copying files to a remote place?
8. What are the essential duties and responsibilities of a Data Scientist in the context of their role and job profile?
9. What is the role and job profile of a Data Steward?
10. Discuss the importance of sharing and licensing in the context of scientific rese

In [2]:
from caching import get_zenodo_ids_from_yaml, get_zenodo_pdfs, download_pdf
import requests
import pdfplumber
from pdf2image import convert_from_bytes
import io

file_url = "https://raw.githubusercontent.com/NFDI4BIOIMAGE/training/main/resources/nfdi4bioimage.yml"
yaml_file = "nfdi4bioimage.yml" 
response = requests.get(file_url)

# Download the current Training Material yaml file from the Git Repository
with open(yaml_file, "wb") as file:
    file.write(response.content)
print(f"File downloaded successfully as {yaml_file}")

# Extract the Zenodo Record IDs
zenodo_ids = get_zenodo_ids_from_yaml(yaml_file)
print(f"Found {len(zenodo_ids)} Zenodo records: {zenodo_ids}")

File downloaded successfully as nfdi4bioimage.yml
Found 38 Zenodo records: ['10008464', '10008465', '10083555', '10654775', '10679054', '10687658', '10793699', '10815329', '10816895', '10886749', '10939519', '10942559', '10970869', '10972692', '10990107', '11031746', '11066250', '11107798', '11265038', '11396199', '11472148', '11474407', '11548617', '12623730', '3778431', '4317149', '4328911', '4330625', '4334697', '4461261', '4630788', '4748510', '4748534', '4778265', '8323588', '8329305', '8329306', '8414318']


In [15]:
import os
import requests

def download_zenodo_pdf(zenodo_record_id, pdf_number=1, save_dir="downloaded_pdfs"):
    """
    Downloads a specific PDF from a Zenodo record and saves it locally.

    Parameters:
    ----------
    zenodo_record_id : str
        Zenodo record ID to fetch PDFs from.
    pdf_number : int, optional
        The number of the PDF to download (1-based index).
    save_dir : str, optional
        Directory where the PDF will be saved.

    Returns:
    --------
    str : Path to the downloaded PDF file.
    """

    # Fetch list of PDFs from Zenodo
    pdf_files = get_zenodo_pdfs(zenodo_record_id)
    if not pdf_files:
        raise ValueError(f"No PDFs found for Zenodo record {zenodo_record_id}")

    # Ensure the requested PDF exists
    if pdf_number > len(pdf_files):
        raise ValueError(f"Requested PDF {pdf_number} not available, only {len(pdf_files)} found.")

    # Get the selected PDF file
    pdf_url = pdf_files[pdf_number - 1]["links"]["self"]
    pdf_filename = (f"zenodo_{zenodo_record_id}_pdf{pdf_number}.pdf")

    # Download and save PDF
    response = requests.get(pdf_url)
    if response.status_code == 200:
        with open(pdf_filename, "wb") as f:
            f.write(response.content)
        print(f" Downloaded PDF saved at: {pdf_filename}")
        return pdf_filename
    else:
        raise ValueError(f"Failed to download PDF (Status Code: {response.status_code})")


In [20]:
zenodo_record_id = "12623730"  
pdf_number = 2  # Change to the desired PDF number

# Step 1: Download PDF
pdf_path = download_zenodo_pdf(zenodo_record_id, pdf_number)

from pdf_utilities import save_images

#save_images("downloaded_images", pdf_path)

 Downloaded PDF saved at: zenodo_12623730_pdf2.pdf


## Test wit zenodo urls


In [51]:
import requests

def get_zenodo_image_urls(record_id):
    """
    Fetches image URLs from a Zenodo record.

    Parameters:
    ----------
    record_id : str
        The Zenodo record ID.

    Returns:
    --------
    list : List of image URLs.
    """
    api_url = f"https://zenodo.org/api/records/{record_id}"
    response = requests.get(api_url)

    if response.status_code != 200:
        raise ValueError(f"Failed to fetch Zenodo record {record_id}")

    record_data = response.json()
    
    # Extract file URLs that are images
    image_urls = [
        file["links"]["self"] for file in record_data.get("files", [])
        if file["key"].lower().endswith((".png", ".jpg", ".jpeg"))
    ]

    return image_urls

# Example usage
zenodo_record_id = "11344132"  # Replace with your Zenodo record ID
image_urls = get_zenodo_image_urls(zenodo_record_id)
print(image_urls)  # List of image URLs


[]
